# Consultancy report analysis
## Motorbike scrambling track
By Gillian Kennedy (B00805416)

## Introduction
Enter text about problem?

## Aim 
Analyse the potential impact of the motorbike scrambling track in the Binevenagh area.

## Objectives
1. Analyse potential impact on local residents
2. Analyse potential impact on environment
3. Analyse potential impact on tourism

## Data Provided
- NI_roads
- NI_rivers
- NI_outline
- ASSI
- AONB
- Landcover
- Gazeteer
- Settlements
- Census population
- Children 0-7
- Pointer data
- new buildings

## Preparing data for analysis

First we need to import the appropriate packages for the analysis.

In [ ]:
#use figures interactively
%matplotlib widget

#importing packages
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import folium
from shapely.geometry import Point, LineString, Polygon
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature

plt.ion() #make plotting interactive

In [ ]:
# Loading all shapefile data
outline = gpd.read_file(os.path.abspath('data_files/NI_outline.shp'))
roads = gpd.read_file(os.path.abspath('data_files/NI_roads.shp'))
rivers = gpd.read_file(os.path.abspath('data_files/NI_rivers.shp'))
settlements = gpd.read_file(os.path.abspath('data_files/settlements_poly.shp'))
gazeteer = gpd.read_file(os.path.abspath('data_files/Gazeteer.shp'))
assi = gpd.read_file(os.path.abspath('data_files/ASSI.shp'))
aonb = gpd.read_file(os.path.abspath('data_files/AONB.shp'))
census = gpd.read_file(os.path.abspath('data_files/census_areas.shp'))
landcover = gpd.read_file(os.path.abspath('data_files/NW_coast_Land_Cover.shp'))

In [ ]:
#create a Universal Transverse Mercator reference system to transform the data
ni_utm = ccrs.UTM(29) 

In [ ]:
# Ensuring all data is on the same projection
outline = outline.to_crs(ni_utm)
roads = roads.to_crs(ni_utm)
rivers = rivers.to_crs(ni_utm)
settlements = settlements.to_crs(ni_utm)
gazeteer = gazeteer.to_crs(ni_utm)
assi = assi.to_crs(ni_utm)
aonb = aonb.to_crs(ni_utm)
census = census.to_crs(ni_utm)
landcover = landcover.to_crs(ni_utm)

In [ ]:
#Creating a map figure
fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ni_utm)

In [ ]:
# Adding layers to the map figure
outline_feature = ShapelyFeature(outline['geometry'], # first argument is the geometry
                                 ni_utm, # second argument is the CRS
                                 edgecolor='k', # set the edgecolor to black
                                 facecolor='w', # set the facecolor to white
                                 zorder=1) # first layer to be drawn on map
ax.add_feature(outline_feature) # add the feature to the map figure

roads_feat = ShapelyFeature(roads['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='firebrick', # set the edgecolor to be firebrick
                            facecolor='firebrick', # set the facecolor to be firebrick
                            linewidth=0.2, # set the linewidth to be 0.2 pt
                            zorder=2) # second layer to be drawn on map
ax.add_feature(roads_feat) # add the feature to the map figure


settlements_feat = ShapelyFeature(settlements['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='gray', # set the edgecolor to be gray
                            facecolor='gray', # set the facecolor to be gray
                            linewidth=1, # set the linewidth to be 0.2 pt
                            zorder=3) # third layer to be drawn on map
ax.add_feature(settlements_feat) # add the feature to the map figure

gazeteer_feat = ax.scatter(gazeteer.geometry.x, gazeteer.geometry.y, # first argument is the geometry
                            transform=ccrs.UTM(zone=29, southern_hemisphere=False), # second argument is the CRS
                            color='yellow', # set color to yellow
                            s= 10, # set size to 10
                            zorder=4) # fourth layer to be drawn on map

xmin, ymin, xmax, ymax = outline.total_bounds # using the boundary of the outline shapefile feature, zoom the map to area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) # because total_bounds gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

# Show map figure
fig


In [ ]:
# Clear map figure
ax.clear()

# Adding layers to map figure
outline_feature = ShapelyFeature(outline['geometry'], # first argument is the geometry
                                 ni_utm, # second argument is the CRS
                                 edgecolor='k', # set the edgecolor to be black
                                 facecolor='w', # set the facecolor to be white
                                 zorder=1) # first layer to be drawn on map
ax.add_feature(outline_feature) # add the feature to the map figure

assi_feat = ShapelyFeature(assi['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='pink', # set the edgecolor to be pink
                            facecolor='pink', # set the facecolor to be pink
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=2) # second layer to be drawn on map
ax.add_feature(assi_feat) # add the feature to the map figure

aonb_feat = ShapelyFeature(aonb['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='purple', # set the edgecolor to be purple
                            facecolor='purple', # set the facecolor to be purple
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=3) # third layer to be drawn on map
ax.add_feature(aonb_feat) # add the feature to the map figure

rivers_feat = ShapelyFeature(rivers['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='blue', # set the edgecolor to be blue
                            facecolor='blue', # set the facecolor to be blue
                            linewidth=0.2, # set the linewidth to be 0.2 pt
                            zorder=4) # fourth layer to be drawn on map
ax.add_feature(rivers_feat) # add the feature to the map figure

landcover_feat = ShapelyFeature(landcover['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='green', # set the edgecolor to be green
                            facecolor='green', # set the facecolor to be green
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=5) # fith layer to be drawn on map
ax.add_feature(landcover_feat) # add the feature to the map figure

census_feat = ShapelyFeature(census['geometry'], # first argument is the geometry
                            ni_utm, # second argument is the CRS
                            edgecolor='orange', # set the edgecolor to be orange
                            facecolor='orange', # set the facecolor to be orange
                            linewidth=1, # set the linewidth to be 1 pt
                            zorder=6) # sixth layer to be drawn on map
ax.add_feature(census_feat) # add the feature to the map figure


xmin, ymin, xmax, ymax = outline.total_bounds # using the boundary of the outline shapefile feature, zoom the map to area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) # because total_bounds gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

# Show map figure
fig 

As can be seen from the two maps, census areas and landcover are they only layers which do not cover the extent of Northern Ireland as they have already been reduced to cover the area of Binevenagh where the motorbike track will be located. Since we are focusing on the area surrounding the track we should clip all the layers to the extent of the study area. 

In [ ]:
# Creating a polygon to represent the study area
study_area = Polygon([(-7.030049, 55.041508), (-7.026122, 55.203187), (-6.633537, 55.19942), (-6.639045, 55.037764)])

#creating geodataframe for study area
study_area_gdf = gpd.GeoDataFrame({'geometry': [study_area]}, crs=4326).to_crs(ni_utm)

# Clearing map figure
ax.clear()

#Adding NI outline and study area to map to show where study is in Northern Ireland
outline_feature = ShapelyFeature(outline['geometry'], # first argument is the geometry
                                 ni_utm, # second argument is the CRS
                                 edgecolor='k', # set the edgecolor to be black
                                 facecolor='w', # set the facecolor to be white
                                 zorder=1) # first layer to be drawn on map
ax.add_feature(outline_feature) # add the feature to the map figure

study_area_gdf.plot(ax=ax, color='none', edgecolor='forestgreen')

xmin, ymin, xmax, ymax = outline.total_bounds # using the boundary of the outline shapefile feature, zoom the map to area of interest
ax.set_extent([xmin-5000, xmax+5000, ymin-5000, ymax+5000], crs=ni_utm) # because total_bounds gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

# Show map figure
fig

In [ ]:
# Clipping layers to study area extent
roads_st = roads.clip(study_area_gdf)
rivers_st = rivers.clip(study_area_gdf)
settlements_st = settlements.clip(study_area_gdf)
gazeteer_st = gazeteer.clip(study_area_gdf)
assi_st = assi.clip(study_area_gdf)
aonb_st = aonb.clip(study_area_gdf)

In [ ]:
# Clearing map figure
ax.clear()

#Adding layers to the map figure
outline.plot(ax=ax, color='none', edgecolor='black')

# Plot one the clipped layers
roads_st.plot(ax=ax, color='gray', edgecolor='black')

xmin, ymin, xmax, ymax = study_area_gdf.total_bounds # using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-1000, xmax+1000, ymin-1000, ymax+1000], crs=ni_utm) # because total_bounds gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

# Show map figure
fig

In [ ]:
#Creating a point to represent the track centre
track_centre = Point(-6.833074, 55.143815)

#Creating a geodataframe for track centre point
track_gdf = gpd.GeoDataFrame({'geometry': [track_centre]}, crs=4326).to_crs(ni_utm)

# Plotting track centre
track_gdf.plot(ax=ax, color='red', markersize=50)

# Show the map
fig


### Preparing Building data

In the cell below the pointer data for the Binevenagh area will be added to the notebook and points will be created from this data.

In [ ]:
df = pd.read_csv('data_files/Binevenagh_Pointer.csv') # read csv data

# create a new geodataframe
buildings = gpd.GeoDataFrame(df,
                             geometry=gpd.points_from_xy(df['X_COR'], df['Y_COR']), 
                             crs=29903).to_crs(ni_utm)


buildings.head()

In [ ]:
# Plotting track centre
buildings.plot(ax=ax, color='darkblue', markersize=2)

# Show the map
fig

The Binevenagh Pointer data is out of date and new buildings have been constructed since the pointer data was collected. A local resident carried out a survey providing GPS points of the most recently built house in the area surrounding the motorbike track.

In [ ]:
print(f"number of buildings: {len(buildings)}")

In [ ]:
df2 = pd.read_csv('data_files/new_buildings.csv') # read csv data

# create a new geodataframe
new_buildings = gpd.GeoDataFrame(df2,
                             geometry=gpd.points_from_xy(df2['easting'], df2['northing']), 
                             crs=29903).to_crs(ni_utm)


new_buildings.head()

In [ ]:
print(f"number of new buildings: {len(new_buildings)}")

In [ ]:
buildings_updated = gpd.GeoDataFrame(pd.concat([buildings, new_buildings], ignore_index=True))

print(f"number of new buildings: {len(buildings_updated)}")

In [ ]:
buildings_updated.head()

### Preparing landcover data

In [ ]:
landcover.head()

In [ ]:
# define landcover types in a list
type = ['broad-leaved woodland', 'coniferous woodland', 'arable horticulture', 'improved grassland', 'neutral grass', 'calcareous grass', 'acid grass', 'bracken', 'dwarf shrub heath', 
        'open dwarf shrub heath', 'bog', 'water (inland)', 'inland bare ground', 'suburban/rural developed', 'continuous urban', 'supra-littoral sediment', 'littoral rock',
        'littoral sediment', 'saltmarsh', 'sea/estuary']

# BHSUB number that corresponds to the type of landcover
values = [1.1, 2.1, 4.2, 5.1, 6.1, 7.1, 8.1, 9.1, 10.1, 10.2, 12.1, 13.1, 16.1, 17.1, 17.2, 19.1, 20.1, 21.1, 21.2, 22.1]

# Category each landcover type falls into
category = ['woodland', 'woodland', 'agricultural (intensive)', 'agricultural (intensive)', 'agricultural (non-intensive)', 'agricultural (non-intensive)', 'agricultural (non-intensive)',
            'semi-natural', 'semi-natural', 'semi-natural', 'semi-natural', 'water', 'non-vegetated', 'built environment', 'built environment', 'non-vegetated', 'non-vegetated', 'non-vegetated',
            'semi-natural', 'water']

landcover_type = dict(zip(values, type)) # create a dict of landcover value/type pairs
landcover_category = dict(zip(values, category)) # create a dict of landcover value/category pairs

In [ ]:
# Add landcover type and landcover category as columns to landcover shapefile
landcover["Landcover_Type"] = landcover["BHSUB"].map(landcover_type)
landcover["Landcover_Category"] = landcover["BHSUB"].map(landcover_category)

# show attribute table of landcover shapefile
landcover.head()

In [ ]:
# Clearing map figure
ax.clear()

#Adding layers to the map figure
outline_feature = ShapelyFeature(outline['geometry'], # first argument is the geometry
                                 ni_utm, # second argument is the CRS
                                 edgecolor='k', # set the edgecolor to be black
                                 facecolor='w', # set the facecolor to be white
                                 zorder=1) # first layer to be drawn on map
ax.add_feature(outline_feature) # add the feature to the map figure

landcover_colors = ['lawngreen', 'palegreen', 'gray', 'orangered', 'saddlebrown', 'aqua', 'darkgreen']

landcover_eco_category = list(landcover.Landcover_Category.unique())
landcover_eco_category = [str(x) for x in landcover_eco_category]  # Convert everything to string!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
landcover_eco_category.sort()

for ii, name in enumerate(landcover_eco_category):
    landcover_feat = ShapelyFeature(landcover.loc[landcover["Landcover_Category"] == name, "geometry"],
                                    ni_utm,
                                    edgecolor='k',
                                    facecolor=landcover_colors[ii],
                                    linewidth=1,
                                    alpha=0.25)
    ax.add_feature(landcover_feat)

xmin, ymin, xmax, ymax = study_area_gdf.total_bounds # using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin-1000, xmax+1000, ymin-1000, ymax+1000], crs=ni_utm) # because total_bounds gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.


# Show map figure
fig

### Preparing census data

For the analysis section of the code we will want to know if the area surrounding the motorbike track has a high proportion of young children. As the noises from the motorbike track my affect the children at bedtime. So we need to join the children data to the census data.

In [ ]:
children = pd.read_csv('data_files/children_0-7.csv') # read csv data

children.head()

In [ ]:
census.head()

In [ ]:
census_updated = census.merge(children, left_on='OA_CODE', right_on='OA')

census_updated.head()

## Analysing Data

There are many buildings surrounding the motorbike track and so many people that could potentially be affected by the noise produced by the track. Before assessing the impact of the track could potntially have on local residents it might be of interest to see how far away the owner of the track lives. The owner of the track lives in a village called Articlave.

In [ ]:
gazeteer.head()

In [ ]:
# Selecting the village Articlave
articlave = gazeteer.query("NAME == 'ARTICLAVE'").geometry.iloc[0]

# calculating the distance of Articlave village to the track
owner_distance = track_gdf.distance(articlave)

distance_value = owner_distance.iloc[0]

print(f'Distance of track to owners house: {distance_value:.2f} meters')

There has been official complaints made about the motorbike track from local residents. The majority of complaints come from people living on the Altikeeragh Road, Ballyhackett Road, and Burrenmore Road. We can use these complaints to estimate how far the noise of the track travels in the area.

In [ ]:
# Calculating the distance of all buildings in the study area from the track
buildings["distance_to_track"] = buildings.geometry.distance(track_gdf.geometry.iloc[0])

# Selecting the roads where complaint were made from
complaints = buildings.query("ROAD in ['ALTIKEERAGH ROAD', 'BALLYHACKETT ROAD', 'BURRENMORE ROAD']")

print(complaints[["ROAD", "distance_to_track"]])

As we can see from the results above most complaints are within 1.2km. However there have also been complaints made by holiday-makers staying at a caravan park at 56 Ballywoodock Road who say noise can be heard from the track in calm weather.

In [ ]:
# Selecting the caravan park the complaint was made from
complaints2 = buildings.query("ROAD in ['BALLYWOODOCK ROAD'] and BUILDING == 56")

print(complaints2[["BUILDING", "ROAD", "distance_to_track"]])

From calculating the distance from the track to the caravn park we now know that the noise from the track can be heard over 2km away.
Another factor to help ascertain the impact of the track on local residents is to calculate the population density of the surrounding are and the percentage of children in the surrounding area.

In [ ]:
census_updated = census_updated.rename(columns={"CHILD_0-7": "CHILD_0_7"})

census_updated.head()

In [ ]:

def population_calculations(census_updated, POPN, CHILD_0_7):
    """
    Calculates area, population density, and percentage of children.

    Parameters:
    census_updated: The polygon dataset (already loaded & projected).
    POPN (str): Column name containing total population values from the census.
    CHILD_0_7 (str): Column name containing number of children aged 0-7.

    Returns:
    census_updated: Updated census_updated with new columns 'area', 'population_density', and 'children_percentage'.
    """
    # Calculate polygon area
    census_updated["area"] = census_updated.geometry.area

    # Calculate population density
    census_updated["pop_density"] = (census_updated[POPN] / census_updated["area"]) * 1_000_000

    # Calculate percentage of children
    census_updated["pc_child"] = (census_updated[CHILD_0_7] / census_updated[POPN]) * 100

    return census_updated

# Example usage:
census_updated = population_calculations(census_updated, "POPN", "CHILD_0_7")

# Print the first few rows
print(census_updated[["POPN", "CHILD_0_7", "area", "pop_density", "pc_child"]])

In [ ]:
# Calculate area for landcover shapefile
landcover["area"] = landcover.geometry.area

landcover.head()

In [ ]:
# Creating a 2km buffer around the track centre
buffer_2km = track_gdf.buffer(2000)

# Plot buffer onto map figure
buffer_2km.plot(ax=ax, facecolor='none', edgecolor='k')

# Show map figure
fig

## Results

### Impact on local residents


In [ ]:
# Plot folium map
m = buffer_2km.explore(fill=False,
                       color='black')
                       

census_updated.explore('pop_density', 
                       m=m,
                       cmap='RdYlBu',
                       legend_kwds={'caption': 'Population density'})

track_args = {
    'm' : m,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)

buildings_args = {
    'm' : m,
    'marker_type': 'circle_marker',
    'popup': True,
    'legend': False,
    'marker_kwds': { 'radius': 2, 'color': 'gray', 'fill': True, 'fill_color': 'gray', 'fillOpacity': 1.0}
}
    

buildings_updated.explore(**buildings_args)


# Show folium map
m

In [ ]:
# Plot folium map
m2 = buffer_2km.explore(fill=False,
                       color='black')
                       

census_updated.explore('pc_child', 
                       m=m2,
                       cmap='RdYlBu',
                       legend_kwds={'caption': 'Percentage of children'})

track_args = {
    'm' : m2,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)

buildings_args = {
    'm' : m2,
    'marker_type': 'circle_marker',
    'popup': True,
    'legend': False,
    'marker_kwds': { 'radius': 2, 'color': 'gray', 'fill': True, 'fill_color': 'gray', 'fillOpacity': 1.0}
}
    

buildings_updated.explore(**buildings_args)


# Show folium map
m2

### Impact on the environment

In [ ]:
# Plot folium map
m2 = buffer_2km.explore(fill=False,
                       color='black')
                       

landcover.explore('category', 
                       m=m2,
                       legend_kwds={'caption': 'Percentage of children'})

track_args = {
    'm' : m2,
    'marker_type': 'marker',
    'popup': False,
    'legend': False,
    'marker_kwds': {'icon': folium.Icon(color='purple', icon='motorcycle', prefix='fa')}
}

track_gdf.explore(**track_args)




# Show folium map
m3